In [1]:
import ReportTimerange as rtr
import ReportMailer as rm
import GuestbookDbConnect as gdb

# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xlsxwriter

# Reporting Period #
* PD - yesterday
* PW - previous week
* PM - previous month
* PQ - previous quarter
* PY - previous calendar year
* YTD year to date
* WTD current week to date
* TODAY today
* P90 - previous 90days

In [2]:
acronyms = [ "PD", "PW", "PM", "PQ", "PY", "YTD", "WTD", "TODAY", "P90"]
friendly = [ "Prior Day", "Prior Week", "Prior Month", "Prior Quarter", "Prior Year", "Year to Date", "Week to Date", "Today", "Previous 90 days"]

def toPeriodFriendly(INTERVAL):
    for n in range(0, len(acronyms)):
      if INTERVAL==acronyms[n]:
         return friendly[n]
    return "Unknown Period"

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

def exit(): raise StopExecution

In [3]:
import os
import pandas as pd

#path = "/home/gbadmin/jupy-notebooks/Reports/inqueue"
#dir_list = os.listdir(path)

#if len(dir_list)>0:
#  filename = "inqueue/{}".format(dir_list[0])
#  print(filename)

#  rep = pd.read_json(filename, lines=True)

#  for _ , row in rep.iterrows():    
#    _INTERVAL = row['interval']
    
#  print(_INTERVAL)
#  # delete the file
#  os.remove(filename)  
#else:
#  print("NOTHING TO DO")
#  exit()

#_INTERVAL = "PY"

path = "/home/gbadmin/jupy-notebooks/Reports/inqueue"
dir_list = os.listdir(path)

if len(dir_list)>0:
  filename = "inqueue/{}".format(dir_list[0])
  print(filename)

  rep = pd.read_json(filename, lines=True)

  for _ , row in rep.iterrows():    
    _INTERVAL = row['interval']
    _TYPE = row['type']
    if (_TYPE=='ALL'):
       print(_INTERVAL)
       print(_TYPE)
       # delete the file
       os.remove(filename)
    else:
      print("Non-ALL Type")
      exit()
       
else:
  print("NOTHING TO DO")
  exit()



inqueue/report-20221017_174706.json
PQ
ALL


_INTERVAL = "PY"

In [4]:
from pandas import ExcelWriter
from pandas import ExcelFile
import xlsxwriter
import datetime

def createSpreadsheetAndMailIt(collections, reportname, recipients, subject, body):
  now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
  filename = reportname + "-" + now + ".xlsx"
  #writer = ExcelWriter(filename)
  writer = ExcelWriter(filename, engine='xlsxwriter')
  workbook = writer.book  
  print("Writing dataframe to Excel file {0}".format(filename))
  for collection in collections:
    print("Writing {} to spreadsheet".format(collection["name"]))
    collection["dataframe"].to_excel(writer, sheet_name=collection["name"], index=False)
    worksheet = writer.sheets[collection["name"]]
    if 'colwidths' in collection:
      colwidths = collection['colwidths']
      print("colwidths={}".format(colwidths))
      for n in range(0, len(colwidths)):
        col = str(chr(65 + n))
        worksheet.set_column('{}:{}'.format(col,col), colwidths[n])
    else:
      worksheet.set_column('A:A', 30)
  writer.save()
  for recipient in recipients: 
    rm.mailer(recipient, subject, body, filename)
  print("Excel/Email Done!")

In [5]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


In [6]:
trange = rtr.timerange(_INTERVAL)
print(trange)

collections = []
summaries = []

collection = {}
collection["name"] = "Outline"

df = pd.DataFrame({"Report Tab":[ \
              "Summary", \
              "Distinct Clients", \
              "Services", \
              "Housing", \
              "Incarcerations", \
              "Gender", \
              "Ethnicity", \
              "Veterans", \
              "New Clients"
                                 ], \
                   "Description":[ \
              "summary totals for the report period", \
              "a count of individuals served by sub-period within the period.", \
              "breakdown by service type and sub-period within the period.", \
              "breakdown by housing conditions and sub-period within the period.", \
              "clients reporting incarceration in previous 90 day period.", \
              "breakdown by gender and sub-period within the period.", \
              "breakdown by ethnicity and sub-period within the period.", \
              "breakdown by veteran status and sub-period within the period.", \
              "a list of clients with first visit occurring in the report period."
                                 ]})

collection["dataframe"] = df 
collection["colwidths"] = [30, 100]
collections.append(collection)



df.head(1000)

[datetime.datetime(2022, 7, 1, 0, 0), datetime.datetime(2022, 9, 30, 23, 59, 59, 999999), 'YYYY-MM/DD WW']


,Report Tab,Description
0,Summary,summary totals for the report period
1,Distinct Clients,a count of individuals served by sub-period wi...
2,Services,breakdown by service type and sub-period withi...
3,Housing,breakdown by housing conditions and sub-period...
4,Incarcerations,clients reporting incarceration in previous 90...
5,Gender,breakdown by gender and sub-period within the ...
6,Ethnicity,breakdown by ethnicity and sub-period within t...
7,Veterans,breakdown by veteran status and sub-period wit...
8,New Clients,a list of clients with first visit occurring i...


## Total distinct clients in report period ##

In [7]:
summary = {}
summary["name"] = "Total distinct clients in report period"

query = "SELECT min(timestamp)AS start, max(timestamp) AS end, \
         COUNT(DISTINCT person_id) AS clients FROM guestbook_personsnapshot \
         WHERE timestamp BETWEEN '{}' AND '{}'".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

summary["count"] = data.iloc[0,2]
summaries.append(summary)
print(summaries)

data.head(100)

SELECT min(timestamp)AS start, max(timestamp) AS end,          COUNT(DISTINCT person_id) AS clients FROM guestbook_personsnapshot          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'
[{'name': 'Total distinct clients in report period', 'count': 342}]


,start,end,clients
0,2022-07-01 11:39:55.276075+00:00,2022-09-30 14:33:13.490389+00:00,342


# Distinct clients by sub-interval within the report period #
* Statistics focused on the breadth of individuals that are served during the time period.

In [8]:
collection = {}
collection["name"] = "Distinct Clients"

query = "SELECT to_char(timestamp,'{}') AS period,  \
         COUNT(DISTINCT person.idperson) AS clients \
         FROM guestbook_personsnapshot snapshot \
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period".format(trange[2], trange[0], trange[1])
print(query)

data = pd.read_sql(query, conn)
#data.head(1000)

persons = data.pivot_table('clients', index=['period']).fillna(0).astype(int).reset_index('period')

collection["dataframe"] = persons 
collection["colwidths"] = [30, 20]
collections.append(collection)
persons.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,           COUNT(DISTINCT person.idperson) AS clients          FROM guestbook_personsnapshot snapshot          JOIN guestbook_person person ON person.idperson=snapshot.person_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY period


,period,clients
0,2022-07/01 26,39
1,2022-07/04 27,25
2,2022-07/05 27,43
3,2022-07/06 27,2
4,2022-07/07 27,35
5,2022-07/08 27,49
6,2022-07/11 28,50
7,2022-07/12 28,42
8,2022-07/13 28,38
9,2022-07/14 28,42


## Total Services delivered in report period ##

In [9]:
query = "SELECT \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE timestamp BETWEEN '{}' AND '{}'  AND service.points<=0 \
         GROUP BY servicename \
         ORDER BY total desc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)


for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = data.iloc[n,0] + " services delivered."
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'  AND service.points<=0          GROUP BY servicename          ORDER BY total desc
[{'name': 'Total distinct clients in report period', 'count': 342}, {'name': 'Meals services delivered.', 'count': 2167}, {'name': 'Shower services delivered.', 'count': 749}, {'name': 'Clothing services delivered.', 'count': 360}, {'name': 'Nurse services delivered.', 'count': 203}, {'name': 'Laundry services delivered.', 'count': 182}, {'name': 'Group Class services delivered.', 'count': 116}, {'name': 'Pastor Consultation services delivered.', 'count': 38}, {'name': 'Housing Assistance

,servicename,total
0,Meals,2167
1,Shower,749
2,Clothing,360
3,Nurse,203
4,Laundry,182
5,Group Class,116
6,Pastor Consultation,38
7,Housing Assistance,35
8,Computer,16
9,Toiletrie,7


# Services delivered by sub-interval within the report period #

In [10]:
collection = {}
collection["name"] = "Services"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         service.name AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' \
#         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND service.points<=0 \
         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])



print(query)

data = pd.read_sql(query, conn)

services = data.pivot_table('total', index=['period'], columns='servicename').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = services 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)

services.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999' AND service.points<=0          GROUP BY period, servicename


servicename,period,Clothing,Computer,Group Class,Housing Assistance,Laundry,Meals,Nurse,Pastor Consultation,Shower,Toiletrie
0,2022-07/01 26,6,0,3,0,1,28,6,1,10,0
1,2022-07/04 27,0,0,0,0,0,25,0,1,7,0
2,2022-07/05 27,7,1,5,0,4,39,2,0,14,0
3,2022-07/07 27,9,0,0,0,3,28,6,1,10,0
4,2022-07/08 27,9,0,2,0,1,35,7,0,15,0
5,2022-07/11 28,10,0,0,0,6,45,11,3,16,0
6,2022-07/12 28,3,0,6,2,3,39,2,0,7,0
7,2022-07/13 28,6,1,0,2,3,35,2,2,13,1
8,2022-07/14 28,12,0,2,3,4,34,3,3,13,1
9,2022-07/15 28,2,0,0,0,3,31,3,2,6,0


# Housing totals within the report period#

In [11]:
query = "SELECT  \
         left(housing.name, strpos(housing.name, '(') - 1)  AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY hresponse".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = " Days spent in " + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           left(housing.name, strpos(housing.name, '(') - 1)  AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999' AND prompt_id=8          GROUP BY hresponse
[{'name': 'Total distinct clients in report period', 'count': 342}, {'name': 'Meals services delivered.', 'count': 2167}, {'name': 'Shower services delivered.', 'count': 749}, {'name': 'Clothing services delivered.', 'count': 360}, {'name': 'Nurse services delivered.', 'count': 203}, {'name': 'Laundry services delivered.', 'count': 182}, {'name': 'Group Class services delivered.', 'count': 116}, {'name': 'Pastor Consultation services delivered.', 'count': 38}, {'name': 'Housing Assistance services delivered.', 'count': 35}, {'name': 'Computer service

,hresponse,total
0,Unstable housing,308
1,Jail or priso,2
2,Sheltered,504
3,Stable housing,177
4,Unsheltered,1323


# Housing by sub-interval within the report period#
* Answers to the prompt 'Where did you spend last night?'

In [12]:
collection = {}
collection["name"] = "Housing"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
#housing = housing.fillna(0)
collection["dataframe"] = housing  
collection["colwidths"] = [30, 30, 30, 30, 30, 30]
collections.append(collection)
housing.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999' AND prompt_id=8          GROUP BY period, hresponse


hresponse,period,Jail or priso,Sheltered,Stable housing,Unsheltered,Unstable housing
0,2022-07/01 26,0,6,6,18,5
1,2022-07/04 27,1,10,1,13,0
2,2022-07/05 27,0,10,3,18,12
3,2022-07/06 27,0,0,0,2,0
4,2022-07/07 27,0,6,3,17,4
5,2022-07/08 27,0,6,1,22,11
6,2022-07/11 28,0,9,3,30,4
7,2022-07/12 28,0,12,2,23,5
8,2022-07/13 28,0,6,3,22,4
9,2022-07/14 28,0,10,5,19,4


# Clients (by name) Reporting Unsheltered Housing #
* more than once in the report period

In [13]:
collection = {}
collection["name"] = "Unsheltered_by_Name"

unsheltered = 5
threshold = 1

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT person.firstname, person.lastname, person.aliasname, \
         COUNT(*) AS unsheltered_nights FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 AND object_id={} \
         GROUP BY person.idperson ORDER BY COUNT(*) desc".format(trange[0], trange[1], unsheltered)

print(query)

data = pd.read_sql(query, conn)
data.head(100)

#housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = data 
collection["colwidths"] = [30, 30, 30, 30, 30, 30]
collections.append(collection)
data.head(1000)

SELECT person.firstname, person.lastname, person.aliasname,          COUNT(*) AS unsheltered_nights FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999' AND prompt_id=8 AND object_id=5          GROUP BY person.idperson ORDER BY COUNT(*) desc


,firstname,lastname,aliasname,unsheltered_nights
0,STEPHANIE,HELMS,STEPH,59
1,AARON,AYERS,AARONA,47
2,MARTY,WEST,0000MARTY_WEST,47
3,CHARLEY,BLISSIT JR,KAKAROTT,46
4,ROBERT,RAMIN,1234ROBERT_RAMIN,46
5,DAVID,STIREWALT,HAWK,43
6,KAYSE,SIERRA,1234KAYSE_SI,39
7,RICHARD,NELTON,0000RICHARD_NELTON,30
8,PAUL,FURR,0000PAUL_FURR,28
9,RANDY,HARRINGTON,RANDYH,28


# Clients Reporting Incarceration in last 90 days #

In [14]:
collection = {}
collection["name"] = "Incarcerations"


#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT person.idperson, person.firstname, person.lastname\
         FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         WHERE survey.object_id=2 AND timestamp BETWEEN '{}' AND '{}' AND prompt_id=4 \
         GROUP BY person.idperson, person.firstname, person.lastname \
         ORDER BY person.idperson".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

#housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = data 
collection["colwidths"] = [30, 30, 30]
collections.append(collection)
data.head(1000)

SELECT person.idperson, person.firstname, person.lastname         FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          WHERE survey.object_id=2 AND timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999' AND prompt_id=4          GROUP BY person.idperson, person.firstname, person.lastname          ORDER BY person.idperson


,idperson,firstname,lastname
0,76,LARRY,LANE
1,169,LARRITT,SHEPPARD
2,221,FRANK,FLEMING
3,281,KIMBERLY,ALEXANDER
4,315,TERRY,GARLAND
5,479,REGINALD,MOODY
6,575,JAMES,NICHOLSON
7,674,DONNIE,VINCENT
8,828,ROBERTL,LYNCH
9,947,KRISTI,JONES


# Gender totals in the report period #

In [15]:
query = "SELECT  \
         gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = "Gender -" + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(100)

SELECT           gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY response
[{'name': 'Total distinct clients in report period', 'count': 342}, {'name': 'Meals services delivered.', 'count': 2167}, {'name': 'Shower services delivered.', 'count': 749}, {'name': 'Clothing services delivered.', 'count': 360}, {'name': 'Nurse services delivered.', 'count': 203}, {'name': 'Laundry services delivered.', 'count': 182}, {'name': 'Group Class services delivered.', 'count': 116}, {'name': 'Pastor Consultation services delivered.', 'count': 38}, {'name': 'Housing Assistance services delivered.', 'count': 35}, {'name': 'Computer services delivered.', 'count': 16}, {'name': 'Toiletrie s

,response,total
0,Female,47
1,Male,116
2,Unknown,179


# Gender by sub-interval within the report period #

In [16]:
collection = {}
collection["name"] = "Gender"

query = "SELECT to_char(timestamp,'{}') AS period, \
         gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

gender = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = gender
collection["colwidths"] = [30, 20, 20, 20]
#collection["colwidths"] = []
collections.append(collection)
gender.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY period, response


response,period,Female,Male,Unknown
0,2022-07/01 26,7,19,13
1,2022-07/04 27,4,12,9
2,2022-07/05 27,7,23,13
3,2022-07/06 27,0,1,1
4,2022-07/07 27,9,15,11
5,2022-07/08 27,6,25,18
6,2022-07/11 28,12,22,16
7,2022-07/12 28,10,20,12
8,2022-07/13 28,11,15,12
9,2022-07/14 28,4,16,22


# Ethnicity totals within the report period #

In [17]:
query = "SELECT  \
         ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)
for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = "Ethnicity -" + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT           ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY response
[{'name': 'Total distinct clients in report period', 'count': 342}, {'name': 'Meals services delivered.', 'count': 2167}, {'name': 'Shower services delivered.', 'count': 749}, {'name': 'Clothing services delivered.', 'count': 360}, {'name': 'Nurse services delivered.', 'count': 203}, {'name': 'Laundry services delivered.', 'count': 182}, {'name': 'Group Class services delivered.', 'count': 116}, {'name': 'Pastor Consultation services delivered.', 'count': 38}, {'name': 'Housing Assistance services delivered.', 'count': 35}, {'name': 'Computer services delivered.', 'count': 16}, {'n

,response,total
0,Black,87
1,Hispanic,2
2,Native American,3
3,Other,4
4,Unknown,161
5,White,85


# Ethnicity by sub-interval within the report period #

In [18]:
collection = {}
collection["name"] = "Ethnicity"

query = "SELECT to_char(timestamp,'{}') AS period, \
         ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

ethnicity = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = ethnicity 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)
ethnicity.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY period, response


response,period,Black,Hispanic,Native American,Other,Unknown,White
0,2022-07/01 26,12,0,1,1,12,13
1,2022-07/04 27,9,0,0,0,7,9
2,2022-07/05 27,17,1,1,0,10,14
3,2022-07/06 27,1,0,0,0,0,1
4,2022-07/07 27,12,0,0,1,9,13
5,2022-07/08 27,16,0,1,2,16,14
6,2022-07/11 28,14,0,0,1,13,22
7,2022-07/12 28,13,0,1,1,10,17
8,2022-07/13 28,13,0,0,1,12,12
9,2022-07/14 28,11,0,0,1,18,12


# Veteran status totals within the report period#

In [19]:
query = "SELECT  \
         yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response \
         ORDER BY response desc".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  if data.iloc[n,0]=="Yes":
    summary["name"] = "Veterans"
  elif data.iloc[n,0]=="No":
    summary["name"] = "Non-veterans"
  else:
    summary["name"] = "Veteran status Unknown"
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY response          ORDER BY response desc
[{'name': 'Total distinct clients in report period', 'count': 342}, {'name': 'Meals services delivered.', 'count': 2167}, {'name': 'Shower services delivered.', 'count': 749}, {'name': 'Clothing services delivered.', 'count': 360}, {'name': 'Nurse services delivered.', 'count': 203}, {'name': 'Laundry services delivered.', 'count': 182}, {'name': 'Group Class services delivered.', 'count': 116}, {'name': 'Pastor Consultation services delivered.', 'count': 38}, {'name': 'Housing Assistance services delivered.', 'count': 35}, {'name': 'Computer services delivered.', 'count'

,response,total
0,Yes,10
1,Unknown,191
2,No,141


# Veteran Status by sub-interval within the report period#

In [20]:
collection = {}
collection["name"] = "Veterans"

query = "SELECT to_char(timestamp,'{}') AS period, \
         yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)
#data.head(1000)



veterans  = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
veterans.columns = ['period', 'non-veteran', 'unknown','veteran']
collection["dataframe"] = veterans 
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)
veterans.head(1000)


SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id          WHERE timestamp BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          GROUP BY period, response


,period,non-veteran,unknown,veteran
0,2022-07/01 26,25,13,1
1,2022-07/04 27,15,9,1
2,2022-07/05 27,29,13,1
3,2022-07/06 27,1,1,0
4,2022-07/07 27,23,11,1
5,2022-07/08 27,29,19,1
6,2022-07/11 28,31,16,3
7,2022-07/12 28,29,12,1
8,2022-07/13 28,24,12,2
9,2022-07/14 28,18,23,1


# New Clients within the report period#
* Clients first appearing in the report interval

In [21]:
collection = {}
collection["name"] = "New Clients"

query = "SELECT firstname, lastname, aliasname, timelinestarttime::date AS startdate FROM guestbook_person \
         WHERE timelinestarttime BETWEEN '{}' AND '{}' \
         ORDER BY timelinestarttime asc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)
collection["dataframe"] = data
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)

data.head(5000)

SELECT firstname, lastname, aliasname, timelinestarttime::date AS startdate FROM guestbook_person          WHERE timelinestarttime BETWEEN '2022-07-01 00:00:00' AND '2022-09-30 23:59:59.999999'          ORDER BY timelinestarttime asc


,firstname,lastname,aliasname,startdate
0,ROBIN,SLIGH,0000ROBIN_SLIGH,2022-07-01
1,MELISSA,HESS,8103MELISSA_HESS,2022-07-01
2,RICHARD,NELTON,0000RICHARD_NELTON,2022-07-05
3,RUTH,HOOKS,0000RUTH_HOOKS,2022-07-07
4,JANET,JACKSON,0000JANET_JACKSON,2022-07-07
5,SELENA,JOHNSON,5635SELENA_JOHNSON,2022-07-08
6,DUSTIN,GRIMES,0000DUSTIN_GRIMES,2022-07-08
7,JOHN,MCCLENDON,9235JOHNBMCCLENDON,2022-07-08
8,CYNTHIA,PHARES,0000CYNTHIA_PHARES,2022-07-11
9,LISANDRO,GOMEZ,0000LISANDRO_GOMEZ,2022-07-13


# Summary Counts #

In [22]:
collection = {}
summ = pd.DataFrame.from_dict(summaries)
collection["name"] = "Summary"
collection["dataframe"] = summ
collection["colwidths"] = [30, 60]
collections.insert(1, collection)
#collections.append(collection)
summ.head(1000)

,count,name
0,342,Total distinct clients in report period
1,2167,Meals services delivered.
2,749,Shower services delivered.
3,360,Clothing services delivered.
4,203,Nurse services delivered.
5,182,Laundry services delivered.
6,116,Group Class services delivered.
7,38,Pastor Consultation services delivered.
8,35,Housing Assistance services delivered.
9,16,Computer services delivered.


In [23]:
_REPORTNAME = _INTERVAL
_SUBJECT = "Opportunity House - Reports for {}".format(toPeriodFriendly(_INTERVAL))
_BODY    = "Spreadsheet (attached) with reports for period {} through {}.".format(trange[0], trange[1])
_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'jahood1@yahoo.com']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'pastor@opphouse.net']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com']

createSpreadsheetAndMailIt(collections, _REPORTNAME, _EMAIL_RECIPIENT, _SUBJECT, _BODY)

Writing dataframe to Excel file PQ-2022-10-17_174758.xlsx
Writing Outline to spreadsheet
colwidths=[30, 100]
Writing Summary to spreadsheet
colwidths=[30, 60]
Writing Distinct Clients to spreadsheet
colwidths=[30, 20]
Writing Services to spreadsheet
colwidths=[30, 20, 20, 20, 20, 20, 20, 20]
Writing Housing to spreadsheet
colwidths=[30, 30, 30, 30, 30, 30]
Writing Unsheltered_by_Name to spreadsheet
colwidths=[30, 30, 30, 30, 30, 30]
Writing Incarcerations to spreadsheet
colwidths=[30, 30, 30]
Writing Gender to spreadsheet
colwidths=[30, 20, 20, 20]
Writing Ethnicity to spreadsheet
colwidths=[30, 20, 20, 20, 20, 20, 20, 20]
Writing Veterans to spreadsheet
colwidths=[30, 30, 30, 30]
Writing New Clients to spreadsheet
colwidths=[30, 30, 30, 30]
Emailer Done!
Excel/Email Done!
